1. Dijkstra算法、分层图最短路
+ 前置知识:
  + 讲解025、026、027-堆结松
  + 讲解032-位图, 用一个整型变量最多可以表示 32 个状态, 并且非常方便、快速讲解059-建图、链式前向星
  + 讲解061-最小生成树 prim算法利用反向索引堆做的优化 强烈推荐看一下讲解062-宽度优先遍历及其扩展
  + 讲解059 讲解065都是【必备】课程有关图的内容, 建议从头开始学习

1. Dijkstra算法及其优化
分层图最短路, 又叫扩点最短路 or BFS

Dijkstra算法、分层图最短路
Dijkstra算法：给定一个源点, 求解从源点到每个点的最短路径长度。单源最短路径算法。适用范围: 有向图、边的权值没有负数

彻底暴力的Dijkstra算法, 不讲、时间复杂度太差、无意义

普通堆实现的Dijkstra算法，最普遍、最常用
3. 算法核心过程:
节点弹出过就忽略
节点没弹出过, 让其它没弹出节点距离变小的记录加入堆
反向索引堆实现的Dijkstra算法, 最快速、最极致
核心在于掌握反向索引堆
对应本节题目1、题目2、题目 3

反向索引堆
Dijkstra算法、分层图最短路
反向索引堆实现的Dijkstra算法, 时间复杂度 $0(m * \log n), n$ 为节点数, $m$ 为边数
1, 准备好反向索引堆, 根据源点到当前点的距离组织小根堆, 可以做到如下操作
    a. 新增记录 ($x$, 源点到 $x$ 的距离)
    b. 当源点到x的距离更新时, 可以进行堆的调整
    c. $x$ 点一旦弹出, 以后忽略 $x$
    d. 弹出堆顶的记录 ( $u$, 源点到 $u$ 的距离)
2, 把(源点, 0)加入反向索引堆, 过程开始
3, 反向索引堆弹出 (u, 源点到 $u$ 的距离), 考察 $u$ 的每一条边, 假设某边去往 $v$, 边权为 $w$
    1) 如果v没有进入过反向索引堆里, 新增记录 ($v$, 源点到 $u$ 的距离 $+w$)
    2) 如果 $v$ 曾经从反向索引堆弹出过, 忽略
    3) 如果 $v$ 在反向索引堆里, 看看源点到 $v$ 的距离能不能变得更小, 如果能, 调整堆; 不能, 忽略
    4) 处理完u的每一条边, 重复步骤3
4, 反向索引堆为空过程结束。反向索引堆里记录了源点到每个节点的最短距离。

 如何实现：
讲解059-建图、链式前向星 + 反向索引堆

	// 链式前向星+反向索引堆的实现
	public static int networkDelayTime2(int[][] times, int n, int s) {
		build(n);
		for (int[] edge : times) {
			addEdge(edge[0], edge[1], edge[2]);
		}
		addOrUpdateOrIgnore(s, 0);
		while (!isEmpty()) {
			int u = pop();
			for (int ei = head[u]; ei > 0; ei = next[ei]) {
				addOrUpdateOrIgnore(to[ei], distance[u] + weight[ei]);
			}
		}
		int ans = Integer.MIN_VALUE;
		for (int i = 1; i <= n; i++) {
			if (distance[i] == Integer.MAX_VALUE) {
				return -1;
			}
			ans = Math.max(ans, distance[i]);
		}
		return ans;
	}

	public static int MAXN = 101;

	public static int MAXM = 6001;

	// 链式前向星
	public static int[] head = new int[MAXN];

	public static int[] next = new int[MAXM];

	public static int[] to = new int[MAXM];

	public static int[] weight = new int[MAXM];

	public static int cnt;

	// 反向索引堆
	public static int[] heap = new int[MAXN];

	// where[v] = -1，表示v这个节点，从来没有进入过堆
	// where[v] = -2，表示v这个节点，已经弹出过了
	// where[v] = i(>=0)，表示v这个节点，在堆上的i位置
	public static int[] where = new int[MAXN];

	public static int heapSize;

	public static int[] distance = new int[MAXN];

	public static void build(int n) {
		cnt = 1;
		heapSize = 0;
		Arrays.fill(head, 1, n + 1, 0);
		Arrays.fill(where, 1, n + 1, -1);
		Arrays.fill(distance, 1, n + 1, Integer.MAX_VALUE);
	}
    	// 链式前向星建图
	public static void addEdge(int u, int v, int w) {
		next[cnt] = head[u];
		to[cnt] = v;
		weight[cnt] = w;
		head[u] = cnt++;
	}

	public static void addOrUpdateOrIgnore(int v, int c) {
		if (where[v] == -1) {
			heap[heapSize] = v;
			where[v] = heapSize++;
			distance[v] = c;
			heapInsert(where[v]);
		} else if (where[v] >= 0) {
			distance[v] = Math.min(distance[v], c);
			heapInsert(where[v]);
		}
	}

	public static void heapInsert(int i) {
		while (distance[heap[i]] < distance[heap[(i - 1) / 2]]) {
			swap(i, (i - 1) / 2);
			i = (i - 1) / 2;
		}
	}

    public static int pop() {
		int ans = heap[0];
		swap(0, --heapSize);
		heapify(0);
		where[ans] = -2;
		return ans;
	}

	public static void heapify(int i) {
		int l = 1;
		while (l < heapSize) {
			int best = l + 1 < heapSize && distance[heap[l + 1]] < distance[heap[l]] ? l + 1 : l;
			best = distance[heap[best]] < distance[heap[i]] ? best : i;
			if (best == i) {
				break;
			}
			swap(best, i);
			i = best;
			l = i * 2 + 1;
		}
	}

	public static boolean isEmpty() {
		return heapSize == 0;
	}

	public static void swap(int i, int j) {
		int tmp = heap[i];
		heap[i] = heap[j];
		heap[j] = tmp;
		where[heap[i]] = i;
		where[heap[j]] = j;
	}

}    

In [ ]:
import heapq

def network_delay_time1(times, n, s):
    # Create an adjacency list for the graph
    graph = [[] for _ in range(n+1)]
    for edge in times:
        graph[edge[0]].append((edge[1], edge[2]))

    # Initialize distances to all nodes as infinity, and set the source distance to 0
    distance = [float('inf')] * (n + 1)
    distance[s] = 0

    # Keep track of visited nodes
    visited = [False] * (n + 1)

    # Create a priority queue and add the source node with distance 0
    heap = [(0, s)]  # (distance, node)

    while heap:
        # Pop the node with the smallest distance from the priority queue
        d, u = heapq.heappop(heap)
        if visited[u]:
            continue

        # Mark the node as visited
        visited[u] = True

        # Update the distance for each neighbor
        for v, w in graph[u]:
            if not visited[v] and distance[u] + w < distance[v]:
                distance[v] = distance[u] + w
                heapq.heappush(heap, (distance[v], v))

    # Return the maximum distance which is the network delay time
    # If there are nodes that weren't reached, return -1
    max_dist = max(distance[1:])
    return max_dist if max_dist != float('inf') else -1

# Example usage:
times = [[2, 1, 1], [2, 3, 1], [3, 4, 1]]
n = 4
s = 2
print(network_delay_time1(times, n, s))
